# 提取信息

需要从公告中提取地理标志产品名称、地理标志产品产地范围等信息。

因为产品实在是太多（4000多个），文件的格式太多太复杂，使用一般的信息提取方法（模式识别）很困难。

所以考虑使用大模型进行提取，提示词见附录，大模型采用``Qwen3-Max``。

> 注意：大模型一次能提取的产品数约为50-60个，因此要分批多次提取。

提取的信息最后整合为一个JSON文件，包含产品名称，产地的省、市、县/区和产品感官特征信息，结果保存在``data/extract``目录下。

示例如下：

-----
```
{
    "product_name": "玉林香蒜",
    "province": "广西壮族自治区",
    "city": "玉林市",
    "districts": [
        "玉州区"
    ],
    "sensory_characteristics": "分一等品、二等品两个等级,各等级质量指标须符合表 1 规定.表 1 香蒜质量指标| 项目|等 级|等 级|等 级|等 级|| 项目|一等品|一等品|二等品|二等品|| 感 观|1. 蒜头形状色泽一致;成熟、清洁、汁浓味香 辣;饱满紧凑,蒜头由 2 至 3 层蒜瓣组成,蒜 瓣基部微紫色,一般有 12 至 20 瓣,排列不整 齐,不散瓣;须根 ≤0.5 ㎝. 2. 无霉烂、发芽、虫蛀、机械伤、皱缩空腔.|1. 蒜头形状色泽一致;成熟、清洁、汁浓味香 辣;饱满紧凑,蒜头由 2 至 3 层蒜瓣组成,蒜 瓣基部微紫色,一般有 12 至 20 瓣,排列不整 齐,不散瓣;须根 ≤0.5 ㎝. 2. 无霉烂、发芽、虫蛀、机械伤、皱缩空腔.|1. 蒜头形状色泽基本一致;成熟、清洁、汁浓味 香辣;饱满紧凑,蒜头由 2 至 3 层蒜瓣组成,蒜 瓣基部微紫色,一般有 12 至 20 瓣,排列不整 齐,不散瓣;须根 ≤0.5 ㎝. 2. 无霉烂、发芽、虫蛀,机械伤、皱缩空腔.|1. 蒜头形状色泽基本一致;成熟、清洁、汁浓味 香辣;饱满紧凑,蒜头由 2 至 3 层蒜瓣组成,蒜 瓣基部微紫色,一般有 12 至 20 瓣,排列不整 齐,不散瓣;须根 ≤0.5 ㎝. 2. 无霉烂、发芽、虫蛀,机械伤、皱缩空腔."
}
```
-----

所使用的提示词如下：

-----

你是一位专业的地理标志产品信息整理专家。请**严格依据我接下来提供的全部官方公告文本内容（包括正文及其所附的所有附件）**，为其中**每一个被批准实施地理标志（或原产地域）保护的产品**提取指定字段，并以 **标准 JSON 格式** 输出一个对象数组（`list of objects`）。每个对象必须且仅包含以下五个字段：

- `product_name`（产品名称）  
- `province`（省）  
- `city`（市）  
- `districts`（县/区，或具体乡镇列表）  
- `sensory_characteristics`（感官特征）

**提取与处理规则（务必逐条遵守）：**

1. **产品识别原则**：  
   - 每份公告**可能包含一个或多个**被保护的地理标志产品（例如标题或正文中明确列出多个产品名）。  
   - 你必须为**每一个独立产品**生成一个 JSON 对象。  
   - 判断依据：公告正文中“现批准自即日起对【XXX】实施……保护”中的【XXX】即为一个产品。若出现多次此类句式，则对应多个产品。
2. **product_name**：  
   - 必须**完全照抄**公告中用于申请保护的产品全称，不得缩写、合并或改写。
3. **province / city / districts**：  
   - **首先查阅公告正文中的“保护范围”部分；若该部分仅说明“见附件X”，则必须从对应附件中提取具体的产地范围描述。**  
   - 若附件或正文中明确写出县级及以上地名（如“南江县”“雷州市”“天台县”“大连市”），则：
     - `districts`：若列出具体乡镇，填入完整乡镇名数组（保留“乡”“镇”后缀）；若仅写“XX县现辖行政区域”，则填 `[ "XX县" ]`；若为“XX市现辖行政区域”且未列县，则填 `[]`。
     - `city` 和 `province`：**即使公告未明说**，也**允许你根据中国现行行政区划常识**推断并填写所属地级市和省级名称（例如：“雷州市” → city: "湛江市", province: "广东省"；“巴马瑶族自治县” → city: "河池市", province: "广西壮族自治区"）。
   - **严禁篡改或扩展公告原文的保护范围**。例如，公告写“伽师县的英买里乡……等13个乡镇”，你不能把 `districts` 写成 `["伽师县"]`，而应列出所有13个乡/镇名（若原文已枚举）；若原文只写“伽师县现辖行政区域”，则 `districts = ["伽师县"]`。
   - 若公告及附件中保护范围表述模糊到无法确定任何县级单位（极罕见），`districts` 可为空数组 `[]`，但仍需尽可能补全省、市。
4. **sensory_characteristics**：  
   - **必须优先查阅公告正文及所有附件**，寻找出现在“感官特色”“感官特征”“感观特征”“外观”“气味及滋味”“品质特色”等小标题下的**纯文字描述**。  
   - 若在**正文或附件中找到相关描述**，则**必须逐字引用原文**，禁止包含理化指标、加工方法等内容。  
   - **仅当公告正文及所有附件均未提供任何感官描述段落**（包括无“外观”“气味”“滋味”等相关文字），**才允许你联网搜索该地理标志产品的权威资料（如国家知识产权局官网、地方政府官网、农业农村部公告等）**，从中**客观、准确地提炼出公认的感官特征描述**，并以简洁、规范的中文写入该字段。  
   - **严禁在公告或附件已有感官描述的情况下自行补充或改写**；仅当**完全缺失**该部分内容时，才可启用联网搜索补充。  
   - 所有补充内容必须基于可靠来源，不得虚构、夸大或主观臆测。
5. **输出格式要求**：  
   - 整体输出必须是**合法、可直接解析的 JSON 数组**。  
   - 每个产品一个对象，顺序按其在公告中出现的先后排列。  
   - 所有字段必须存在，值可为空（如 `districts: []`）。  
   - **绝对禁止**包含任何额外内容：无解释、无注释、无序号、无“```json”、无总结语。
